### Игра "Racer"
Это простая 2D-игра, в которой игрок управляет персонажем, собирая монеты и избегая врагов.
Основной игровой процесс включает в себя перемещение игрока, генерацию монет и врагов, а также
подсчет очков.


## components

In [ ]:
import pygame
import random
from components import create_coin, draw_player, draw_coins, collect_coins, draw_coin_count, move_enemy

# инициализация game, этот файл у меня управляет логикой игры. Он импортирует функции из components.py и запускает
# игровой процесс: движение игрока, сбор монет, увеличение скорости врагов и другие аспекты
pygame.init()

# устанавливаем цвета
WHITE = (255, 255, 255)
RED = (255, 0, 0)
BLUE = (0, 0, 255)

# частота кадров (fps как в кс го но намного тормознейXD)
clock = pygame.time.Clock()
FPS = 30

# переменные игры
length, width = 1000, 600  # размеры экрана
player_pos = pygame.Rect(400, 300, 50, 50)  # игрок как прямоугольник (положение и размер)
player_speed = 5  # скорость игрока
coins = []  # список монет
collected_coins = 0  # количество собранных монет
coin_radius = 10  # радиус монеты
enemy_speed = 2  # скорость врага
enemy_pos = pygame.Rect(random.randint(0, length-50), random.randint(0, width-50), 50, 50)  # враг

# инициализация экрана
screen = pygame.display.set_mode((length, width))
screen.fill(WHITE)  # заполняем экран белым фоном
running = True  # флаг, управляющий игрой

# главный игровой цикл
while running:
    for event in pygame.event.get():  # обрабатываем все события
        if event.type == pygame.QUIT:  # если окно закрыто
            running = False  # завершаем игру

    # управление движением игрока
    keys = pygame.key.get_pressed()  # получаем состояние клавиш
    if keys[pygame.K_LEFT] and player_pos.left > 0:  # если нажата стрелка влево и игрок не выходит за границы
        player_pos.x -= player_speed
    if keys[pygame.K_RIGHT] and player_pos.right < length:  # если нажата стрелка вправо
        player_pos.x += player_speed
    if keys[pygame.K_UP] and player_pos.top > 0:  # если нажата стрелка вверх
        player_pos.y -= player_speed
    if keys[pygame.K_DOWN] and player_pos.bottom < width:  # если нажата стрелка вниз
        player_pos.y += player_speed

    # случайное создание монет без лог. посл.
    if random.random() < 0.05:  # 5% шанс на создание монеты каждый кадр
        coin = create_coin(length, width, coin_radius)
        coins.append(coin)  # добавляем монету в наш список

    # Сбор монет игроком
    collected_coins += collect_coins(player_pos, coins)  # kolichestvo sozdannyh monet

    # увеличение скорости врага каждое количество монет
    if collected_coins >= 5 and collected_coins % 5 == 0:
        enemy_speed += 1  # увеличиваем скорость врага

    # перемещение врага
    enemy_pos, enemy_speed = move_enemy(enemy_pos, enemy_speed, length)

    # сделать видимым или оботражение все элементы игры
    screen.fill(WHITE)  # Очищаем экран

    # отрисовываем игрока
    draw_player(screen, player_pos)

    # отрисовываем монеты
    draw_coins(screen, coins)

    # отображаем количество собранных монет
    draw_coin_count(screen, collected_coins, length)

    # делаем видимым врага врага
    pygame.draw.rect(screen, RED, enemy_pos)  # рисуем врага как прямоугольник

    pygame.display.flip()  # Обновляем экран

    # ограничиваем количество кадров в секунду
    clock.tick(FPS)

pygame.quit()  # Закрытие после выхода из игры


## game

In [ ]:
"""
Это игра Змейка, где игрок, студент, учитель или любой человек управляет змейкой, 
которая двигается по экрану, съедает еду и растет. 
Игрок должен избегать столкновений с стенами или с самой собой, иначе игра заканчивается.
"""

import pygame
import random
from pygame.locals import *
#Мы подключаем библиотеку pygame, которая нужна для создания графики и анимации. 
#Также подключаем random, чтобы случайным образом генерировать еду для змеи.

pygame.init()
#Эта команда запускает все необходимые компоненты библиотеки Pygame для работы с графикой, звуком и т. д.

# Настройка экрана игры чисто хуже hd чтобы не лагалXD
screen_width = 600
screen_height = 400
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("Snake Game")


white = (255, 255, 255)
black = (0, 0, 0)
red = (255, 0, 0)
green = (0, 255, 0)

# шрифт для текста
font_small = pygame.font.SysFont("Verdana", 20)

# настройки змеи
block_size = 20  # Размер блока змеи
snake_speed = 10  # Начальная скорость змеи

# основной шрифт для отображения текста
font = pygame.font.SysFont(None, 25)

# зис фанкшин для рисования змеи
def draw_snake(snake_list):
    for block in snake_list:
        pygame.draw.rect(screen, green, [block[0], block[1], block_size, block_size])

# зис фанкшин нужен для отображения сообщения на экране
def message(msg, color):
    screen_text = font.render(msg, True, color)
    screen.blit(screen_text, [screen_width / 6, screen_height / 3])


def generate_food():
    food_x = round(random.randrange(0, screen_width - block_size) / 20.0) * 20.0
    food_y = round(random.randrange(0, screen_height - block_size) / 20.0) * 20.0
    return food_x, food_y


def game_loop(snake_speed):
    game_over = False
    game_close = False

    snake_list = []  
    snake_length = 1  
    snake_x = screen_width / 2  
    snake_y = screen_height / 2  
    snake_x_change = 0  
    snake_y_change = 0  

    # Настройки еды
    food_x, food_y = generate_food()  # Генерируем еду

    # начальный счет и уровень
    score = 0
    level = 1

    # таймер для обновления экрана
    clock = pygame.time.Clock()

    # основной игровой цикл
    while not game_over:
        # экран окончания игры
        while game_close == True:
            screen.fill(white)
            message("You lost! Press Q to quit or C to play again", red)
            pygame.display.update()

            # обработка событий на экране окончания игры
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game_over = True
                    game_close = False
                if event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_q:  # Нажатие Q для выхода из игры
                        game_over = True
                        game_close = False
                    if event.key == pygame.K_c:  # Нажатие C для новой игры
                        game_loop(snake_speed)

        # Обработка движения змеи
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game_over = True
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_LEFT:
                    snake_x_change = -block_size
                    snake_y_change = 0
                elif event.key == pygame.K_RIGHT:
                    snake_x_change = block_size
                    snake_y_change = 0
                elif event.key == pygame.K_UP:
                    snake_y_change = -block_size
                    snake_x_change = 0
                elif event.key == pygame.K_DOWN:
                    snake_y_change = block_size
                    snake_x_change = 0

      
        snake_x += snake_x_change
        snake_y += snake_y_change

        # проверка на столкновение с границами экрана
        if snake_x >= screen_width or snake_x < 0 or snake_y >= screen_height or snake_y < 0:
            game_close = True

        # проверка на столкновение с едой
        if snake_x == food_x and snake_y == food_y:
            score += 1  # увеличиваем счет
            snake_length += 1  # увеличиваем длину змеи
            food_x, food_y = generate_food()  # генерируем новую еду

            # увеличиваем уровень каждый раз, когда съедено 3 еды
            if score % 3 == 0:
                level += 1
                snake_speed += 2  # увеличиваем скорость змеи с каждым уровнем

        # рисуем фон и еду
        screen.fill(white)
        pygame.draw.rect(screen, red, [food_x, food_y, block_size, block_size])

        # добавляем голову змеи в список
        snake_head = []
        snake_head.append(snake_x)
        snake_head.append(snake_y)
        snake_list.append(snake_head)

        # удаляем хвост змеи, если она слишком длинная, так скажем режем)
        if len(snake_list) > snake_length:
            del snake_list[0]

        # Проверка на столкновение змеи с собой
        for block in snake_list[:-1]:
            if block == snake_head:
                game_close = True

     
        draw_snake(snake_list)   # для прорисовки змеи на экране

        
        score_text = font_small.render("Score: " + str(score), True, black)
        level_text = font_small.render("Level: " + str(level), True, black)
        screen.blit(score_text, (10, 10))
        screen.blit(level_text, (10, 30))# Счет уровень показывается блягодоаря этой команде

      
        pygame.display.update()  # нужна для обновление экрана

        
        clock.tick(snake_speed) # регулируем скорость игры в зависимости от текущего уровня

    # ииии завершение столь мощной игры:(
    pygame.quit()
    quit()


game_loop(snake_speed)
